In [41]:

# 1. Importing Libraries
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [42]:
# 2. Loading the MNIST Dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 3. Data Preprocessing
# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# 4. One-Hot Encoding
# One-hot encode the labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [43]:
# 5. Custom Dense Layer
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation):
        super(CustomDenseLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)

    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        return self.activation(z)


In [44]:
# Step 6: Define Neural Network Architecture
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Input layer for MNIST images
    CustomDenseLayer(128, tf.nn.relu),  # Custom dense layer with 128 units and ReLU activation
    CustomDenseLayer(10, tf.nn.softmax)  # Custom dense layer with 10 units (output layer) and softmax activation
])

In [45]:
# 7. Custom Loss Function (custom_sparse_categorical_crossentropy)
def custom_sparse_categorical_crossentropy(y_true, y_pred):
    return -tf.reduce_mean(tf.math.log(tf.reduce_sum(y_true * y_pred, axis=-1) + 1e-10))

In [46]:

# 8. Custom Accuracy Metric (custom_accuracy)
def custom_accuracy(y_true, y_pred):
    correct_predictions = tf.equal(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1))
    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))


In [47]:
# 9. Model Compilation
model.compile(optimizer='adam', loss=custom_sparse_categorical_crossentropy, metrics=[custom_accuracy])

# 10. Model Training
model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.3207 - custom_accuracy: 0.9089 - val_loss: 0.1719 - val_custom_accuracy: 0.9532
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1381 - custom_accuracy: 0.9608 - val_loss: 0.1219 - val_custom_accuracy: 0.9646
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0927 - custom_accuracy: 0.9727 - val_loss: 0.1057 - val_custom_accuracy: 0.9678
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0692 - custom_accuracy: 0.9793 - val_loss: 0.0894 - val_custom_accuracy: 0.9719
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0524 - custom_accuracy: 0.9842 - val_loss: 0.0878 - val_custom_accuracy: 0.9737


In [48]:
# 11. Model Evaluation
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

313/313 [==============================] - 1s 2ms/step - loss: 0.0795 - custom_accuracy: 0.9762
Test accuracy: 97.62%
